# Config stuff

In [3]:

import ConnectionConfig as cc
from delta import DeltaTable
from datetime import datetime

In [4]:
cc.setupEnvironment()
spark = cc.startLocalCluster("dimSalesIncrementalLoad")
spark.getActiveSession()

## Incremental load

After the sales Rep dimension is filled for the first time, the logic to update the dimension has to be handled differently. A change of a record in the source system has to be handled as a change in the dimension. The SCD2 logic is used to handle this.

The SCD2 implementation requires a more complex transformation to correctly handle changes in the source files. For detailed information consult the comments in the code.
### Setting the parameters
The timestamp of the job is used to set the scd_end date of the previous record and the scd_start date of the new record.


In [5]:
run_timestamp =datetime.now() #The job runtime is stored in a variable

### Read existing dimension

In [6]:
dt_dimSalesRep = DeltaTable.forPath(spark,".\\spark-warehouse\\dimSalesRep")

dt_dimSalesRep.toDF().createOrReplaceTempView("dimSalesRep_current")

#DEBUG CODE TO SHOW CONTENT OF DIMENSION
spark.sql("select salesRepID, name, office, salesRepSK, md5  from dimSalesRep_current ").show()

+----------+-------------+-------------+--------------------+--------------------+
|salesRepID|         name|       office|          salesRepSK|                 md5|
+----------+-------------+-------------+--------------------+--------------------+
|         5|     T. Mosby|       Berlin|8f21bff7-22a1-465...|947579dec8084039e...|
|         6|   H. Simpson|       Berlin|ae01bec1-1761-441...|d636d1b0685650b34...|
|         7|   B. Stinson|San Fransisco|826364a5-ff1d-425...|e726b2d8dc0cf9a6f...|
|         8|L. Hofstadter|     Brussels|bebc8d7a-36d9-4c4...|a2bbe52f8274b0f08...|
|         9|    S. Cooper|     Brussels|be2de78a-b299-43f...|d85c73c9d03df0002...|
|        10| F. Underwood|     Brussels|a96cc010-9031-408...|44cd1a6d596b05688...|
|        11|     W. White|     New York|c3c36918-9efc-45d...|f9ea69ce2aa4482b4...|
|        12| T. Lannister|     New York|f550df5f-38ab-482...|3259a471f9816d7c3...|
|        13|      M. Ross|       London|92611993-20ad-478...|d0faf94c1bbe2d4a7...|
|   

### Read source table


##### 1 READ SOURCE TABLE
Creating dataframe with source table (from operational system). Transformed to the dimension format.
The surrogate key is a uuid to be sure it's unique.
md5 hash is used to identify changes in the source table.
A view is created of the resulting dataframe to make it available for the next step.



In [7]:
cc.set_connectionProfile("tutorial_op")

#a. Reading from a JDBC source
df_operational_sales_rep = spark.read \
    .format("jdbc") \
    .option("driver" , cc.get_Property("driver")) \
    .option("url", cc.create_jdbc()) \
    .option("dbtable", "salesrep") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "salesRepID") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 20) \
    .load()

df_operational_sales_rep.createOrReplaceTempView("operational_sales_rep")

#b. Transforming the source to the dimension format
df_dim_sales_rep_new = spark.sql( "select uuid() as source_salesRepSK, \
                                        salesRepId as source_salesRepId, \
                                        name as source_name, \
                                        office as source_office, \
                                        md5(concat( name, office)) as source_md5 \
                                    from operational_sales_rep")

df_dim_sales_rep_new.createOrReplaceTempView("dimSalesRep_new")

#DEBUG CODE TO SHOW CONTENT OF SOURCE
#df_dim_sales_rep_new.printSchema()
#df_dim_sales_rep_new.show()
spark.sql("select * from dimSalesRep_new").show()
#df_dim_sales_rep.write.format("delta").mode("overwrite").saveAsTable("dimSalesRep")


+--------------------+-----------------+-------------+-------------+--------------------+
|   source_salesRepSK|source_salesRepId|  source_name|source_office|          source_md5|
+--------------------+-----------------+-------------+-------------+--------------------+
|965df168-4d51-441...|                1|      R. Zane|       Berlin|1f8cbbc272a33dcc1...|
|dbfce4df-0ae8-423...|                2|   P. Chapman|       Berlin|14b094c31bf9e4149...|
|f15b658c-5922-476...|                3|     F. Crane|     New York|0715f05df18a3a794...|
|910bb144-5d36-47f...|                4|    R. Geller|     New York|6212c0ce01f144d66...|
|942819a8-79ed-486...|                5|     T. Mosby|       Berlin|947579dec8084039e...|
|5f05b4ef-19f7-462...|                6|   H. Simpson|       Berlin|d636d1b0685650b34...|
|bf45fa3a-5d59-4f3...|                7|   B. Stinson|San Fransisco|e726b2d8dc0cf9a6f...|
|1cbea65f-0ec6-4b2...|                8|L. Hofstadter|     Brussels|a2bbe52f8274b0f08...|
|d0fcf349-


##### 2 DETECT CHANGES
Dataframe to identify SCD2 changed rows.
First a join between SOURCE (operational system) and DIMENSION (dwh) is performed
   The md5 hash is used to identify differences.
   The list contains:
       - updated source-rows (the join finds a rowand the md5 is different)  and
       - new source-rows (the leftouter join does not find a row in the dimension (dwh.salesRepId is null)
       

In [8]:

detectedChanges=spark.sql(f"select * \
                          from dimSalesRep_new source \
                          left outer join dimSalesRep_current dwh on dwh.salesRepID == source.source_salesRepId and dwh.current == true \
                          where dwh.salesRepId is null or dwh.md5 <> source.source_md5")

detectedChanges.createOrReplaceTempView("detectedChanges")

#DEBUG CODE TO SHOW CONTENT OF DETECTED CHANGES
detectedChanges.show()


+-----------------+-----------------+-----------+-------------+----------+----------+----------+----+------+---------+-------+---+-------+
|source_salesRepSK|source_salesRepId|source_name|source_office|source_md5|salesRepSK|salesrepid|name|office|scd_start|scd_end|md5|current|
+-----------------+-----------------+-----------+-------------+----------+----------+----------+----+------+---------+-------+---+-------+
+-----------------+-----------------+-----------+-------------+----------+----------+----------+----+------+---------+-------+---+-------+




##### 3 TRANSOFRM TO UPSERTS
Before union: Every updated and new source-row requires the insertion of a new record in the SCD2 dimension. This new records starts at the runtime of the job and ends at the end of time (2100-12-12). Current is set to true.
Updated source-rows also require an update of the existing scd-fields. The scd_end date of the existing record is set to the runtime of the job. Current is set to false

In the next step, rows without mergeKey will be inserted in the dimension table and rows with mergekey will be updated in the dimension

In [9]:

df_upserts = spark.sql(f"select source_salesRepSK as salesRepSK,\
                                source_salesRepId as salesRepID,\
                                source_name as name,\
                                source_office as office,\
                                to_timestamp('{run_timestamp}') as scd_start, \
                                to_timestamp('2100-12-12','yyyy-MM-dd') as scd_end,\
                                source_md5 as md5,\
                                true as current\
                        from  detectedChanges\
                        union \
                        select  salesRepSK,\
                                salesRepId,\
                                name,\
                                office,\
                                scd_start,\
                                to_timestamp('{run_timestamp}') as scd_end,\
                                md5, \
                                false \
                                from detectedChanges \
                        where current is not null")

df_upserts.createOrReplaceTempView("upserts")

In [10]:

#DEBUG CODE TO SHOW CONTENT OF UPSERTS
spark.sql("select * from upserts").show()

+----------+----------+----+------+---------+-------+---+-------+
|salesRepSK|salesRepID|name|office|scd_start|scd_end|md5|current|
+----------+----------+----+------+---------+-------+---+-------+
+----------+----------+----+------+---------+-------+---+-------+




#### PERFORM MERGE DIMSALESREP AND UPSERTS
merge looks for a matching dwh.salesRepID (in the dimension) for mergeKey
   - when a match is found (the dimension table contains a row where its salesRepId corresponds with one of the mergekeys)  -> perform update of row to close the period and set current to "false"
   - when no match is found (there is no salesRepID in the dimension because the mergeKey is null) -> perform an insert with the data from the updserts table (from the source). The scd-start is filled with the run_timestamp)

In [13]:
spark.sql("MERGE INTO dimSalesRep_current AS target \
          using upserts AS source ON target.salesRepID = source.salesRepID and source.current = false and target.current=true \
          WHEN MATCHED THEN UPDATE SET scd_end = source.scd_end, current = source.current  \
          WHEN NOT MATCHED THEN INSERT (salesRepSK, salesRepId, name, office, scd_start, scd_end, md5, current) values (source.salesRepSK, source.salesRepId, source.name, source.office, source.scd_start, source.scd_end, source.md5, source.current)")\

#DEBUG CODE TO SHOW CONTENT OF DIMENSION
dt_dimSalesRep.toDF().sort("salesRepID", "scd_start").show(100)


+--------------------+----------+-------------+-------------+-------------------+-------------------+--------------------+-------+
|          salesRepSK|salesrepid|         name|       office|          scd_start|            scd_end|                 md5|current|
+--------------------+----------+-------------+-------------+-------------------+-------------------+--------------------+-------+
|66a4faaf-1655-4bb...|         1|      R. Zane|       Berlin|1999-01-01 00:00:00|2100-12-12 00:00:00|1f8cbbc272a33dcc1...|   true|
|67f698be-a1e7-46f...|         2|   P. Chapman|       Berlin|1999-01-01 00:00:00|2100-12-12 00:00:00|14b094c31bf9e4149...|   true|
|daedb4af-6a2c-4a0...|         3|     F. Crane|     New York|1999-01-01 00:00:00|2100-12-12 00:00:00|0715f05df18a3a794...|   true|
|92c0a4bf-79c3-4d9...|         4|    R. Geller|     New York|1999-01-01 00:00:00|2100-12-12 00:00:00|6212c0ce01f144d66...|   true|
|b455c302-a6a3-422...|         5|     T. Mosby|       Berlin|1999-01-01 00:00:00|21

## Delete the spark session

In [14]:
spark.stop()